# 不分詞性蒐集字頻(去除括號與詞性)

In [61]:
import glob,codecs,re
import _uniout
import pymongo
from pymongo import MongoClient

def wordcount(findlist):
    words_freq={}
    for word in findlist:
        subterm =re.sub(u'兩造|離婚|被告|原告|新台幣|新臺幣|[\.$「」（）)(、。：，第條款項○○○一二三四五六七八九十萬甲乙丙丁戊壹貳叄肆伍陸柒捌玖拾佰仟萬億元於年月日1234567890]','',word)
        #if word not in punctions:
        if len(subterm)>1:
            if subterm not in words_freq:
                words_freq[subterm] = 1
            else:
                words_freq[subterm] += 1
    #for ele in words_freq:
        #print ele,words_freq[ele]
    return words_freq

client = pymongo.MongoClient('localhost',27017)
db = client.test2
collection = db['plaintiff_sinica_fre']

for filename in glob.glob('E:\\jurydata\\plaintiff_sinica\\*.txt'):
    f = codecs.open(filename,'r','utf-8')
    content = f.read()
    pk = re.search(r'([^\\/]*)\.txt',filename).group(1)
    #print pk
    mm = re.findall(u'　(\S*?)\(',content)
    words_freq = wordcount(mm)
    if len(words_freq)>0:
        words_freq.update({'_id':pk})
        add_mongo = db.plaintiff_sinica_fre.insert(words_freq)
    else:
        words_freq = {'_id':pk,'None':''}
        add_mongo = db.plaintiff_sinica_fre.insert(words_freq)
    #for ele in words_freq:
        #print ele,words_freq[ele]


# 用Stopwords遮掉廢詞(手動版)

In [32]:
import codecs,re
import _uniout
import pymongo
from pymongo import MongoClient

client = pymongo.MongoClient('localhost',27017)
db = client.cutsentence

for cur in db.sinica_annual.find({u'_id':104}):
    dic_loc = cur

dic_strip = {}
for ele in dic_loc:
    dic_strip.update({ele.strip():dic_loc[ele]})

# import stopwords to list
with codecs.open('./cutTerms/stopwords1.txt','r' ,'utf-8') as f :
    readstopwords = f.readlines() #從txt檔讀取每行的停止詞
    f.close()
with codecs.open('./cutTerms/stopwords2.txt','r' ,'utf-8') as f :
    readstopwords2 = f.readlines() #從txt檔讀取每行的停止詞
    f.close()

stopwords = []
for word in readstopwords:
    stopwords.append(word.strip())#從每個停止詞中，去除該死的換行符號
for word2 in readstopwords2:
    stopwords.append(word2.strip())#從每個停止詞中，去除該死的換行符號

# filte the mergewords by stopwords
dic_filter = {}
for ele in dic_strip:
    if ele not in stopwords:
        if dic_strip[ele]>5:
            if re.search(',',ele) is None:
                dic_filter.update({ele:dic_strip[ele]})



In [33]:
import operator
import codecs
# sorted by frequence and convert into list
dic_sorted = sorted(dic_filter.iteritems(),key=operator.itemgetter(1),reverse=True) #change words_freq from dict to list 

for ele in dic_sorted:
    #print type(ele[0])
    #print type(str(ele[1]).decode('utf8'))
    f= codecs.open('./cutTerms/filter_104.txt','a+','utf-8')
    con = ele[0] + '\t' + str(ele[1]).decode('utf8') + '\n'
    f.write(con)
    f.close()

#print _uniout.unescape(str(ele),'utf-8')



# input merged document and output splitted by comma

In [27]:
import glob
import codecs,re
#for ff in glob.glob('./cutTerms/sinicut_location/*.txt'):
#    print ff

#dir_path = './cutTerms/sinicut_location/filter_chayi.txt'
for dir_path in glob.glob('./cutTerms/sinicut_annual/*.txt'):
    filename = re.search('(\w+)\.txt',dir_path).group(1)
    f_old = codecs.open(dir_path,'r','utf-8')
    f_new = codecs.open('./cutTerms/sinicut_newannual/%s.txt'%(filename),'a+','utf-8')
    content = f_old.readlines()
    f_old.close()
    for ele in content:
        w =re.sub('\t',',',ele)
        f_new.write(w)
    f_new.close()

# TF-IDF

In [121]:
import glob
import codecs,re,math,operator

dir_path = glob.glob('./cutTerms/sinicut_newlocation/*txt')

def idf(word,dir_path):
    D = len(dir_path)
    allDoclist = []
    for path in dir_path:
        f = codecs.open(path,'r','utf-8')
        content = f.readlines()
        for ele in content:
            allDoclist.append(ele.split(','))
    dic ={}
    for words in allDoclist:
        if words[0] not in dic:
            dic[words[0]] =1
        else:
            dic[words[0]]+=1
    return math.log(float((D*1.00)/float(dic[word])))

for txtfile in dir_path:
    f = codecs.open(txtfile,'r','utf-8')
    content = f.readlines()
    #print len(content)
    wordlist = []
    for ele in content:
        wordlist.append(ele.split(','))
    N=0
    for ele in wordlist:
        N += int(ele[1])
    tfidflist = {}
    for ele in wordlist:
        tfidflist.update({ele[0]:float(ele[1])/N*idf(ele[0],dir_path)})

    words_freq = sorted(tfidflist.iteritems(),key=operator.itemgetter(1),reverse=True) #change words_freq from dict to list 
    print re.search('(\w+)\.txt',txtfile).group(1)
    print '======================================'
    for ele in range(0,20):
        print words_freq[ele][0],words_freq[ele][1]
    print '======================================'

filter_chayi
入出國 0.0401266671847
聖馬爾定 0.0190028330857
道院 0.0070010437684
升學 0.00600089465863
FACEBOOK 0.00600089465863
媳婦 0.00594069246052
逃妻 0.00501583339809
海峽 0.00414448649517
緬甸 0.00362772029003
流產 0.00343400517878
婆婆 0.00326889075676
外勞 0.00317425525378
印尼國 0.00296476266957
家暴 0.0023982312323
廣西省 0.00235053651491
監護權 0.00232024904575
手術 0.00217833377737
自殺 0.00208767263236
印尼籍 0.00208767263236
腦震盪 0.00203047391802
filter_chunhua
女巫 0.00751419925779
難重圓 0.00601135940623
飲料店 0.00563564944335
海洛因 0.00426537014013
高職 0.00413280959179
教唱 0.0037570996289
烤漆 0.0037570996289
稚女 0.00344717386542
娘家人 0.00344717386542
肄業 0.00338138966601
病史 0.00338138966601
游手好閒 0.00337992324956
學費 0.0033579187614
毒癮 0.00331120621725
家暴 0.0032760183038
酒癮 0.00300567970312
痛心疾首 0.00300567970312
彩券行 0.00300567970312
婆婆 0.00291643284053
監護權 0.00280852342209
filter_hsinchu
芎林鄉 0.0133686219258
橫山鄉 0.0125330830555
負債表 0.0108620053147
避孕 0.0100264664444
活存 0.00919092757402
德州 0.00919092757402
創業 0.00784532714748
股票

IndexError: list index out of range

In [120]:
import glob
import codecs,re,math,operator

dir_path = glob.glob('./cutTerms/sinicut_newlocation/*txt')

def idf(word,dir_path):
    D = len(dir_path)
    allDoclist = []
    for path in dir_path:
        f = codecs.open(path,'r','utf-8')
        content = f.readlines()
        for ele in content:
            allDoclist.append(ele.split(','))
    dic ={}
    for words in allDoclist:
        if words[0] not in dic:
            dic[words[0]] =1
        else:
            dic[words[0]]+=1
    print dic[word]
    return math.log(float((D*1.00)/float(dic[word])))

idf(u'毆打',dir_path)

19


0.0